In [2]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import tensorflow as tf 
import matplotlib.pyplot as plt
import glob as gb   # to retrieve file names or pathnames
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/umegshamza/handwritten-digits-images-dataset")

Skipping, found downloaded files in ".\handwritten-digits-images-dataset" (use force=True to force download)


In [5]:
train="handwritten-digits-images-dataset/handwritten-digits-images-dataset/dataset/training/"

In [6]:
import PIL
from PIL import Image,ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  #truncate-shorten the duration/low truncated images
os.listdir(train)
for file in os.listdir(train):
  f_img = train + file
  for digit in os.listdir(f_img):
    t_img= f_img + '/' + digit
    img = Image.open(t_img)
    img = img.resize((32,32))
    img.save(t_img)

In [7]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20,
                             shear_range=10,
                             samplewise_center=True,
                             samplewise_std_normalization= True,
                             height_shift_range=10,
                             width_shift_range=30,
                             horizontal_flip=True,
                             vertical_flip=True,
                             validation_split=0.2
                             )
train_generator = datagen.flow_from_directory('handwritten-digits-images-dataset/handwritten-digits-images-dataset/dataset/training/',
                                              target_size=(32,32),
                                              subset='training',
                                              shuffle=shuffle)

val_generator = datagen.flow_from_directory('handwritten-digits-images-dataset/handwritten-digits-images-dataset/dataset/training/',
                                            target_size=(32,32),
                                            subset='validation',
                                            shuffle=shuffle)

Found 48013 images belonging to 10 classes.
Found 11997 images belonging to 10 classes.


In [8]:
pre_trained=tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights="imagenet",
    input_shape=(32,32,3)
)

In [9]:
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import layers

In [10]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

x = tf.keras.layers.Flatten()(pre_trained.output)
x = tf.keras.layers.Dense(10, activation='softmax')(x)
       
model = Model( pre_trained.input, x) 

print(model.summary())
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
reduce_lr = ReduceLROnPlateau(patience=0,verbose=1,monitor="val_acc")
early_stop = EarlyStopping(patience=2,verbose=1,monitor="val_loss")
filepath="efficient_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True,mode='max')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 32, 32, 3)    0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 16, 16, 32)   864         ['normalization[0][0]']          
                                                                                              

 lization)                                                                                        
                                                                                                  
 block3b_drop (Dropout)         (None, 4, 4, 48)     0           ['block3b_project_bn[0][0]']     
                                                                                                  
 block3b_add (Add)              (None, 4, 4, 48)     0           ['block3b_drop[0][0]',           
                                                                  'block3a_project_bn[0][0]']     
                                                                                                  
 block4a_expand_conv (Conv2D)   (None, 4, 4, 192)    9216        ['block3b_add[0][0]']            
                                                                                                  
 block4a_expand_bn (BatchNormal  (None, 4, 4, 192)   768         ['block4a_expand_conv[0][0]']    
 ization) 

 ization)                                                                                         
                                                                                                  
 block4c_expand_activation (Act  (None, 2, 2, 384)   0           ['block4c_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block4c_dwconv2 (DepthwiseConv  (None, 2, 2, 384)   3456        ['block4c_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block4c_bn (BatchNormalization  (None, 2, 2, 384)   1536        ['block4c_dwconv2[0][0]']        
 )                                                                                                
          

                                                                                                  
 block5b_activation (Activation  (None, 2, 2, 672)   0           ['block5b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5b_se_squeeze (GlobalAver  (None, 672)         0           ['block5b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5b_se_reshape (Reshape)   (None, 1, 1, 672)    0           ['block5b_se_squeeze[0][0]']     
                                                                                                  
 block5b_se_reduce (Conv2D)     (None, 1, 1, 28)     18844       ['block5b_se_reshape[0][0]']     
          

 agePooling2D)                                                                                    
                                                                                                  
 block5d_se_reshape (Reshape)   (None, 1, 1, 672)    0           ['block5d_se_squeeze[0][0]']     
                                                                                                  
 block5d_se_reduce (Conv2D)     (None, 1, 1, 28)     18844       ['block5d_se_reshape[0][0]']     
                                                                                                  
 block5d_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5d_se_reduce[0][0]']      
                                                                                                  
 block5d_se_excite (Multiply)   (None, 2, 2, 672)    0           ['block5d_activation[0][0]',     
                                                                  'block5d_se_expand[0][0]']      
          

                                                                                                  
 block6a_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block6a_se_reduce[0][0]']      
                                                                                                  
 block6a_se_excite (Multiply)   (None, 1, 1, 672)    0           ['block6a_activation[0][0]',     
                                                                  'block6a_se_expand[0][0]']      
                                                                                                  
 block6a_project_conv (Conv2D)  (None, 1, 1, 192)    129024      ['block6a_se_excite[0][0]']      
                                                                                                  
 block6a_project_bn (BatchNorma  (None, 1, 1, 192)   768         ['block6a_project_conv[0][0]']   
 lization)                                                                                        
          

                                                                                                  
 block6c_drop (Dropout)         (None, 1, 1, 192)    0           ['block6c_project_bn[0][0]']     
                                                                                                  
 block6c_add (Add)              (None, 1, 1, 192)    0           ['block6c_drop[0][0]',           
                                                                  'block6b_add[0][0]']            
                                                                                                  
 block6d_expand_conv (Conv2D)   (None, 1, 1, 1152)   221184      ['block6c_add[0][0]']            
                                                                                                  
 block6d_expand_bn (BatchNormal  (None, 1, 1, 1152)  4608        ['block6d_expand_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 block6f_expand_conv (Conv2D)   (None, 1, 1, 1152)   221184      ['block6e_add[0][0]']            
                                                                                                  
 block6f_expand_bn (BatchNormal  (None, 1, 1, 1152)  4608        ['block6f_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6f_expand_activation (Act  (None, 1, 1, 1152)  0           ['block6f_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6f_dwconv2 (DepthwiseConv  (None, 1, 1, 1152)  10368       ['block6f_expand_activation[0][0]
 2D)      

                                                                                                  
 block6h_expand_activation (Act  (None, 1, 1, 1152)  0           ['block6h_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6h_dwconv2 (DepthwiseConv  (None, 1, 1, 1152)  10368       ['block6h_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block6h_bn (BatchNormalization  (None, 1, 1, 1152)  4608        ['block6h_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block6h_a

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=val_generator,
                              epochs=20,
                              callbacks=[checkpoint,reduce_lr,early_stop],
                              verbose=1,steps_per_epoch=5)

C:\Users\yuvarani\AppData\Local\Temp\ipykernel_20088\713149863.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,
